<a href="https://colab.research.google.com/github/Nowitkiewicz/Projects/blob/main/Project_no_4_loan_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project_no.4 -> Prediction of a loan grant using Random Forest Classifier.
After first prediction with model default settings, I will use GridSearchCV to tune a model.

Dataset downloaded from:

https://www.kaggle.com/datasets/devzohaib/eligibility-prediction-for-loan

***Data dictionary:***

Variable	Description

Loan_ID - 	Unique Loan ID

Gender -	Male/ Female

Married	Applicant - married (Y/N)

Dependents	- Number of dependents

Education	Applicant - Education (Graduate/ Under Graduate)

Self_Employed	- Self employed (Y/N)

ApplicantIncome	- Applicant income

CoapplicantIncome	- Coapplicant income

LoanAmount	- Loan amount in thousands

LoanAmountTerm	- Term of loan in months

Credit_History	- credit history meets guidelines

Property_Area	Urban/ Semi Urban/ Rural

Loan_Status	(Target)-  Loan approved (Y/N)

In [5]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
from google.colab import files

uploaded = files.upload()
# Enter path to downloaded file.

Saving Loan_Data.csv to Loan_Data (2).csv


In [46]:
path = input('Please enter the downloaded file name ')

Please enter the downloaded file name Loan_Data.csv


In [48]:
Loan_df = pd.read_csv(path)


In [49]:
Loan_df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [50]:
#General info on data in set:
Loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [51]:
#Info about quantity of null values:
Loan_df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

As seen there are some missing values in set,so to begin EDA we'll start to fill missing values.

In [52]:
Loan_df['Gender'].value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [53]:
# As we see there is big advantage of males over females,so I decide to fill missing data with "male" status.
Loan_df['Gender'] = Loan_df['Gender'].fillna('Male')

In [54]:
#Preprocessing next column with NaN values.
Loan_df['Married'].value_counts()


Yes    398
No     213
Name: Married, dtype: int64

In [55]:
#In 'Married' column I decide to fill NaN with 'Yes' value.
Loan_df['Married'] = Loan_df['Married'].fillna('Yes')

In [56]:
#Preprocessing 'Dependents' to see NaN values:
Loan_df['Dependents'].value_counts()

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [57]:
#Decision is to fill NaN with 0 value:
Loan_df['Dependents'] = Loan_df['Dependents'].fillna(0)

In [58]:
#Preprocessing 'Self_Employed' to see NaN values:
Loan_df['Self_Employed'].value_counts()

No     500
Yes     82
Name: Self_Employed, dtype: int64

In [59]:
#Decision is to fill NaN with 'No' value:
Loan_df['Self_Employed'] = Loan_df['Self_Employed'].fillna('No')

In [60]:
#Preprocessing 'LoanAmount' to see NaN values:
Loan_df['LoanAmount'].value_counts()

120.0    20
110.0    17
100.0    15
160.0    12
187.0    12
         ..
240.0     1
214.0     1
59.0      1
166.0     1
253.0     1
Name: LoanAmount, Length: 203, dtype: int64

In [61]:
#Because there are many different values in 'LoanAmount' column,
#decision is to fill NaN with mean of all values:
Loan_df['LoanAmount'] = Loan_df['LoanAmount'].fillna(np.mean(Loan_df['LoanAmount']))

In [62]:
#Preprocessing 'LoanAmount' to see NaN values:
Loan_df['Loan_Amount_Term'].value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
240.0      4
84.0       4
120.0      3
60.0       2
36.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [63]:
#Because there are many different values in 'LoanAmount' column,
#decision is to fill NaN with mean of all values:
Loan_df['Loan_Amount_Term'] = Loan_df['Loan_Amount_Term'].fillna(np.mean(Loan_df['Loan_Amount_Term']))

In [64]:
#Preprocessing 'LoanAmount' to see NaN values:

Loan_df['Credit_History'].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [65]:
#Because of the big advantage of '1' value the decision is to fill NaN with '1.0' value:
Loan_df['Credit_History'] = Loan_df['Credit_History'].fillna(1.0)

In [66]:
#Check of data in Loan_df:
Loan_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             614 non-null    object 
 2   Married            614 non-null    object 
 3   Dependents         614 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      614 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [67]:
# To apply data into machine learning model it is necessary to have only numeric type of data in the set.
# So our next goal is to change non numeric data into numeric type.
for label, content in Loan_df.items():
  if pd.api.types.is_object_dtype(content):
    Loan_df[label] = content.astype('category').cat.codes + 1

In [68]:
#Check of data in Loan_df:
Loan_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    int16  
 1   Gender             614 non-null    int8   
 2   Married            614 non-null    int8   
 3   Dependents         614 non-null    int8   
 4   Education          614 non-null    int8   
 5   Self_Employed      614 non-null    int8   
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         614 non-null    float64
 9   Loan_Amount_Term   614 non-null    float64
 10  Credit_History     614 non-null    float64
 11  Property_Area      614 non-null    int8   
 12  Loan_Status        614 non-null    int8   
dtypes: float64(4), int16(1), int64(1), int8(7)
memory usage: 29.5 KB


In [69]:
#Now when all of our data are filled and numeric we can begin to prepare set to fit into ML model.
#Our goal is to analize if a person has a loan granted or not.

In [70]:
#Check of data in Loan_df:
Loan_df['Loan_Status'].value_counts()

2    422
1    192
Name: Loan_Status, dtype: int64

In [71]:
from sklearn.preprocessing import StandardScaler

In [74]:
#Split into train and target features:
scaler = StandardScaler()
scaler.fit_transform(Loan_df)
#scaler.transform(y)
X = Loan_df.drop('Loan_Status',axis=1)
y = Loan_df['Loan_Status']


In [75]:
#Import ML tools:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)

In [76]:
#import Classifier model:
from sklearn.ensemble import RandomForestClassifier
#Instantiate Random Forest Classifier as Clf 
Clf = RandomForestClassifier()


In [77]:
#Fit training data to Clf model 
Clf.fit(X_train,y_train)

RandomForestClassifier()

In [78]:
#Checking results of trained model prediction...
# Loan granted --> 2
# Loan not granted -->1
Clf.predict(X_test)

array([2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2], dtype=int8)

In [79]:
#Evaluating model accuracy on a test data:
Clf.score(X_test,y_test) # Percentage of people with loan granted.

0.7886178861788617